In [17]:
from MT1D import MT1DProblem, MT1DSurvey, MT1DSrc, ZxyRx, Survey, AppResPhaRx
from SimPEG import Maps, DataMisfit
from scipy.constants import mu_0
import numpy as np

In [18]:
rxloc = np.r_[0.]
srcloc = np.r_[0.]
frequency = np.logspace(-3, 2, 25)
rxType = "AppredPha"

if rxType == "Zxy":
    rx = ZxyRx(rxloc, component="both", frequency=frequency)
    rxList = [rx]
elif rxType == "AppredPha":
    rx = AppResPhaRx(rxloc, component="both", frequency=frequency)
    rxList = [rx]
    
src = MT1DSrc(rxList, loc=srcloc)
survey = MT1DSurvey([src])
mesh = survey.setMesh(sigma=0.01, max_depth_core=5000., 
                      ncell_per_skind=10, n_skind=2, 
                      core_meshType="log", max_hz_core=1000.)

sigma = np.ones(mesh.nC) * 0.01

prob = MT1DProblem(mesh, sigmaMap=Maps.ExpMap(mesh), verbose=False)
prob.pair(survey)

>> Smallest cell size = 50 m
>> Padding distance = 316227 m
>> # of padding cells 17
>> # of core cells cells 16


In [19]:
%%time
f = prob.fields(np.log(sigma))
pred = survey.dpred(np.log(sigma), f=f)

CPU times: user 78.1 ms, sys: 3.87 ms, total: 82 ms
Wall time: 81.9 ms


In [20]:
pred.reshape((survey.nFreq, 2))

array([[ 101.07140343,   44.00702818],
       [  99.99861914,   44.88870214],
       [ 100.76432908,   45.03799151],
       [ 100.85849973,   44.99541676],
       [ 100.83384236,   44.99220846],
       [ 100.82661407,   44.99061823],
       [ 100.81695931,   44.98827299],
       [ 100.80453347,   44.98545938],
       [ 100.788524  ,   44.98216293],
       [ 100.76787493,   44.97847183],
       [ 100.74133486,   44.9746596 ],
       [ 100.70767831,   44.97132496],
       [ 100.66635939,   44.96956054],
       [ 100.61891973,   44.97104088],
       [ 100.57119343,   44.97774406],
       [ 100.53503627,   44.99087612],
       [ 100.525714  ,   45.00898525],
       [ 100.55063387,   45.02703881],
       [ 100.59608298,   45.04008388],
       [ 100.63728772,   45.0503667 ],
       [ 100.67241737,   45.06582463],
       [ 100.71795883,   45.08973994],
       [ 100.77759329,   45.12353317],
       [ 100.8532251 ,   45.17226884],
       [ 100.94952624,   45.24363091]])

In [21]:
from SimPEG import Tests
def derChk(m):
    return [survey.dpred(m), lambda mx: prob.Jvec(m, mx)]
Tests.checkDerivative(derChk, np.log(sigma), plotIt=False, num=3, eps=1e-20, dx=np.log(sigma)*2)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    7.784e+02     3.156e+02      nan
 1   1.00e-02    4.875e+01     2.276e+00      2.142
 2   1.00e-03    4.671e+00     3.173e-02      1.856
========================= PASS! =========================
You are awesome.



True

In [22]:
survey.dobs = pred
dmis = DataMisfit.l2_DataMisfit(survey)
m0 = np.log(sigma)*2.

SimPEG.DataMisfit.l2_DataMisfit assigning default eps of 1e-5 * ||dobs||


In [23]:
Tests.checkDerivative(
    lambda m: [dmis(m), dmis.deriv(m)],
    m0,
    plotIt=False,
    num=4,
    dx = m0*1.
)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    2.129e+08     1.349e+08      nan
 1   1.00e-02    8.536e+06     7.298e+05      2.267
 2   1.00e-03    7.875e+05     6.921e+03      2.023
 3   1.00e-04    7.813e+04     6.917e+01      2.000
========================= PASS! =========================
You are awesome.



True

In [24]:
v = np.random.rand(mesh.nC)
w = np.random.rand(pred.shape[0])
wtJv = w.dot(prob.Jvec(m0, v))
vtJtw = v.dot(prob.Jtvec(m0, w))
passed = np.abs(wtJv - vtJtw) < 1e-10
print('Adjoint Test', np.abs(wtJv - vtJtw), passed)

('Adjoint Test', 0.0, True)
